# Climate Data from Home System

In [1]:
from erddapy import ERDDAP
import pandas as pd
import datetime

# for secondary/derived parameters
from metpy.units import units
import metpy.calc as mpcalc

In [2]:
server_url = 'http://raspberrypi.local:8080/erddap'
#server_url = 'http://192.168.2.3:8080/erddap'

In [3]:
e = ERDDAP(server=server_url)
df = pd.read_csv(e.get_search_url(response='csv', search_for='MoonFlower'))
print(df['Dataset ID'].values)

['tempest_moonflower_wx' 'channel_1027974_thingspeak'
 'channel_1037066_thingspeak' 'channel_1314759_thingspeak'
 'channel_1047747_thingspeak' 'channel_843357_thingspeak' 'channel_rpi'
 'MFPurpleAir_Primary_RT' 'allDatasets']


In [4]:
dataset_id='tempest_moonflower_wx'

try:
    d = ERDDAP(server=server_url,
        protocol='tabledap',
        response='csv'
    )
    d.dataset_id=dataset_id
except HTTPError:
    print('Failed to generate url {}'.format(dataset_id))

try:
    df_m = d.to_pandas(
                index_col='time (UTC)',
                parse_dates=True,
                skiprows=(1,)  # units information can be dropped.
                )
    df_m.sort_index(inplace=True)
    df_m.columns = [x[1].split()[0] for x in enumerate(df_m.columns)]

except:
    print(f"something failed in data download {dataset_id}")
    pass

df_m.drop(columns=['device_id', 'bucket_step_minutes', 'wind_lull','wind_interval'],inplace=True)
#stats are all utc driven - but we really want local daily values
df_m=df_m.tz_convert('US/Pacific')

# calculations of various parameters... metpy?
# HDD/CDD, dewpointTemp

In [5]:
df_m['dewpointTemp']=mpcalc.dewpoint_from_relative_humidity(df_m.temperature.values * units.degC,
                                                            df_m.humidity.values * units.percent)
#wetbulb from metpy had issues

In [6]:
df_m['SLP']=df_m.pressure.values * (1+((1013.25/df_m.pressure.values)**((287.05*0.0065)/9.80665)) * (0.0065*87.3)/288.15)**(9.80665/(287.05*0.0065))

In [7]:
df_daily_max = df_m.resample('D').max()
df_daily_min = df_m.resample('D').min()
df_daily_ave = df_m.resample('D').mean()
df_daily_total = df_m.resample('1T').mean().resample('D').sum()
df_m.sample()

,type,wind_avg,wind_gust,wind_dir,pressure,temperature,humidity,lux,uv,solar_radiation,...,strike_distance,strike_count,battery,report_interval,local_daily_precip,precip_final,local_daily_precip_final,precip_analysis_type,dewpointTemp,SLP
time (UTC),,,,,,,,,,,,,,,,,,,,,
2020-08-25 01:47:00-07:00,obs_st,0.0,0.0,0.0,1004.9,9.4,93.0,0.0,0.0,0.0,...,0.0,0.0,2.62,1.0,0.0,NaN,NaN,0.0,8.325825,1015.361362


In [8]:
use_current_month = True

if use_current_month:
    current_month = datetime.datetime.now().month
else:
    current_month = 7

current_month_grid_data=pd.DataFrame()
current_month_grid_data = df_daily_max[df_daily_max.index.month==current_month].temperature
current_month_grid_data = pd.concat([current_month_grid_data,
                                     df_daily_min[df_daily_min.index.month==current_month].temperature.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].temperature.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].dewpointTemp.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].SLP.round(1),
                                     df_daily_total[df_daily_total.index.month==current_month].solar_radiation.round(0),
                                     df_daily_max[df_daily_max.index.month==current_month].uv.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].wind_avg.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].wind_dir.astype(int),
                                     df_daily_max[df_daily_max.index.month==current_month].wind_gust.round(1)
                                    ],axis=1)
current_month_grid_data.columns=('max_temperature','min_temperature','mean_temperature','mean_dewpoint','mean SLP','total_solar_radiation','max_uv_index','average speed','average dir','max gust')

In [9]:
current_month_grid_data['station_id'] = 'tempest'

In [10]:
#this should go to erddap
current_month_grid_data.to_csv(f'Data/MoonflowerTempest_2020{str(current_month).zfill(2)}.csv')

In [11]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['color: red' if v else '' for v in is_max]

def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_min = s == s.min()
    return ['color: blue' if v else '' for v in is_min]

current_month_grid_data.drop('station_id',axis=1).style.apply(highlight_max).apply(highlight_min).format("{:.2f}")


,max_temperature,min_temperature,mean_temperature,mean_dewpoint,mean SLP,total_solar_radiation,max_uv_index,average speed,average dir,max gust
time (UTC),,,,,,,,,,
2021-03-01 00:00:00-08:00,10.60,3.10,6.80,5.60,1020.60,14766.00,1.90,0.30,162.00,2.50
2021-03-02 00:00:00-08:00,6.90,2.80,5.00,4.00,1020.00,90092.00,1.60,0.20,103.00,3.00
2021-03-03 00:00:00-08:00,13.20,1.30,6.80,4.00,1011.50,165964.00,2.30,0.30,128.00,3.80
2021-03-04 00:00:00-08:00,12.10,1.60,6.10,5.20,1016.60,141945.00,2.60,0.50,167.00,7.00
2021-03-05 00:00:00-08:00,9.90,5.80,7.20,6.70,1014.40,75285.00,1.60,0.50,182.00,6.30
2021-03-06 00:00:00-08:00,18.60,3.50,6.60,4.00,1016.70,54711.00,2.00,1.00,196.00,5.70
2021-03-07 00:00:00-08:00,7.40,1.30,4.70,2.90,1019.90,87025.00,5.70,1.30,193.00,6.00
2021-03-08 00:00:00-08:00,12.10,-0.40,5.00,0.80,1018.40,135962.00,3.40,0.40,175.00,3.80
2021-03-09 00:00:00-08:00,11.70,1.30,5.30,1.70,1017.50,103514.00,4.40,0.50,151.00,4.00


In [12]:
### need to manage daily records, monthly records, alltime records

## repeat for each sensor on property

choose month or use current (starting with data subsetting)

In [13]:
if use_current_month:
    constraints = {
        'time>=': datetime.datetime.now().strftime('%Y-%m-01T00:00:00Z'),
    }
else:
    constraints = {
        'time>=': '2020-01-01T00:00:00Z',
        'time<=': '2027-02-10T00:00:00Z',
    }

In [15]:
alldatasets=['channel_1314759_thingspeak',
            'channel_1027974_thingspeak',
            'channel_1037066_thingspeak',
            'channel_1047747_thingspeak',
            'channel_843357_thingspeak',
            'channel_rpi']
df_all = {}

for dataset_id in alldatasets:
    try:
        d = ERDDAP(server=server_url,
            protocol='tabledap',
            response='csv'
        )
        d.dataset_id=dataset_id
        d.constraints=constraints
        
    except HTTPError:
        print('Failed to generate url {}'.format(dataset_id))

    try:
        df_m = d.to_pandas(
                    index_col='time (UTC)',
                    parse_dates=True,
                    skiprows=(1,)  # units information can be dropped.
                    )
        df_m.sort_index(inplace=True)
        df_m.columns = [x[1].split()[0] for x in enumerate(df_m.columns)]

    except:
        print(f"something failed in data download {dataset_id}")
        pass

    #stats are all utc driven - but we really want local daily values
    df_m=df_m.tz_convert('US/Pacific')
    df_all.update({dataset_id:df_m})
    # calculations of various parameters... metpy?
    # HDD/CDD, dewpointTemp

something failed in data download channel_rpi


In [16]:
for k,v in enumerate(df_all):
    print(df_all[v].keys())

Index(['entry_id', 'temperature', 'RH_Percent'], dtype='object')
Index(['entry_id', 'temperature'], dtype='object')
Index(['entry_id', 'temperature', 'RH_Percent'], dtype='object')
Index(['entry_id', 'temperature_internal', 'temperature_external',
       'RH_Percent'],
      dtype='object')
Index(['entry_id', 'temperature', 'RH_Percent', 'Barotemperature', 'SLP'], dtype='object')
Index(['entry_id', 'temperature', 'RH_Percent', 'Barotemperature', 'SLP'], dtype='object')


In [19]:
if use_current_month:
    current_month = datetime.datetime.now().month
else:
    current_month = 7
    
for k,v in enumerate(df_all):
    print(df_all[v].keys())
    df_daily_max = df_all[v].resample('D').max()
    df_daily_min = df_all[v].resample('D').min()
    df_daily_ave = df_all[v].resample('D').mean()
    if ('RH_Percent' in df_all[v].keys()) and ('temperature' in df_all[v].keys()) and (not 'Barotemperature' in df_all[v].keys()):
        print(f"processing {v} :0")
        current_month_grid_data=pd.DataFrame()
        current_month_grid_data = df_daily_max[df_daily_max.index.month==current_month].temperature
        current_month_grid_data = pd.concat([current_month_grid_data,
                                             df_daily_min[df_daily_min.index.month==current_month].temperature.round(1),
                                             df_daily_ave[df_daily_ave.index.month==current_month].temperature.round(1),
                                             df_daily_ave[df_daily_ave.index.month==current_month].RH_Percent.round(1),
                                            ],axis=1)
        current_month_grid_data.columns=('max_temperature','min_temperature','mean_temperature','mean_humidity')
        current_month_grid_data['station_id'] = v    
        current_month_grid_data.to_csv(f'Data/{v}_2020{str(current_month).zfill(2)}.csv')
    elif (not 'RH_Percent' in df_all[v].keys()) and ('temperature' in df_all[v].keys()) and (not 'Barotemperature' in df_all[v].keys()):
        print(f"processing {v} :1")
        current_month_grid_data=pd.DataFrame()
        current_month_grid_data = df_daily_max[df_daily_max.index.month==current_month].temperature
        current_month_grid_data = pd.concat([current_month_grid_data,
                                             df_daily_min[df_daily_min.index.month==current_month].temperature.round(1),
                                             df_daily_ave[df_daily_ave.index.month==current_month].temperature.round(1),
                                            ],axis=1)
        current_month_grid_data.columns=('max_temperature','min_temperature','mean_temperature')
        current_month_grid_data['station_id'] = v
        
        #this should go to erddap
        current_month_grid_data.to_csv(f'Data/{v}_2020{str(current_month).zfill(2)}.csv')
    elif ('RH_Percent' in df_all[v].keys()) and ('temperature' in df_all[v].keys()) and ('Barotemperature' in df_all[v].keys()):
        print(f"processing {v} :2")
        current_month_grid_data=pd.DataFrame()
        current_month_grid_data = df_daily_max[df_daily_max.index.month==current_month].Barotemperature
        current_month_grid_data = pd.concat([current_month_grid_data,
                                             df_daily_min[df_daily_min.index.month==current_month].Barotemperature.round(1),
                                             df_daily_ave[df_daily_ave.index.month==current_month].Barotemperature.round(1),
                                            ],axis=1)
        current_month_grid_data.columns=('max_temperature','min_temperature','mean_temperature')
        current_month_grid_data['station_id'] = v
        
        #this should go to erddap
        current_month_grid_data.to_csv(f'Data/{v}_2020{str(current_month).zfill(2)}.csv')
    else:
        print(f"passing {v} :3")
        pass
    
    print(f'{v}')
current_month_grid_data.drop('station_id',axis=1).style.apply(highlight_max).apply(highlight_min).format("{:.2f}")


Index(['entry_id', 'temperature', 'RH_Percent'], dtype='object')
processing channel_1314759_thingspeak :0
channel_1314759_thingspeak
Index(['entry_id', 'temperature'], dtype='object')
processing channel_1027974_thingspeak :1
channel_1027974_thingspeak
Index(['entry_id', 'temperature', 'RH_Percent'], dtype='object')
processing channel_1037066_thingspeak :0
channel_1037066_thingspeak
Index(['entry_id', 'temperature_internal', 'temperature_external',
       'RH_Percent'],
      dtype='object')
passing channel_1047747_thingspeak :3
channel_1047747_thingspeak
Index(['entry_id', 'temperature', 'RH_Percent', 'Barotemperature', 'SLP'], dtype='object')
processing channel_843357_thingspeak :2
channel_843357_thingspeak
Index(['entry_id', 'temperature', 'RH_Percent', 'Barotemperature', 'SLP'], dtype='object')
processing channel_rpi :2
channel_rpi


,max_temperature,min_temperature,mean_temperature
time (UTC),,,
2021-03-01 00:00:00-08:00,15.80,12.90,13.60
